In [ ]:


class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 2
        gap.NGEN = 2
        # The population of genes is 4
        gap.MU = 4
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]
       
    
        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        some_tuples = (vmpop, pop, invalid_ind, pf)
        return pop[0], some_tuples
        #return vmpop, pop, invalid_ind, pf
        
    def _get_optimization_parameters(self, some_tuples):
        vmpop = some_tuples[0]
        # Your specific unpacking of tuples that _optimize returns
        scores = vmpop[0].score
        pop = some_tupes[1]
        errors = -pop[0].fitness
        parameters = vmpop[0].attrs
        #parameters,errors,scores,_ = zip(*some_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        The Class users version of optimize
        where details are hidden in _optimizae
        '''

        # Do optimization including repeated calls to judge
        model, some_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(some_tuples)
        # Maybe rebuild the original model
        # (i.e. restore the true class from the virtual version)

        # Your code keeps parameter sets and associated scores
        # All the organizing stuff
        
        
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.

        models = [model.__class__(p) for p in parameters]

        # Make a ScoreArray (which is basically a pandas dataframe)
        path = sciunit.ScoreArray(models,scores=scores)
        return model, path, some_tuples

    
t = Test()

import model_parameters as modelp
import numpy as np
import utilities as outils
model = outils.model

model,path,some_tuples = t.optimize(model,modelp)   

Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
<module 'model_parameters' from '/home/jovyan/mnt/neuronunit/tests/model_parameters.py'>
0 0.000991890585525
1 39.7464755275
2 0.327308128489
3 -4.11798993043e-09
4 -30.0602981152
5 -60.3193338093
6 9.1897734523e-05
7 -56.2144635172
8 0.0729517191958
9 -67.2254200683
0 0.000772348729074
1 32.8833838063
2 0.547692036355
3 -4.7207288213e-09
4 -41.4820867624
5 -63.1979060557
6 0.000102335826126
7 -57.1787452753
8

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


 0.187370818106
9 -56.8703714406
0 0.00132587503902
1 36.6427899322
2 0.359360862453
3 -3.58973571481e-09
4 -38.1232448327
5 -72.8016542803
6 0.000104024060643
7 -58.5080419106
8 0.100825775998
9 -55.6816293568
0 0.00102709349519
1 31.2281656237
2 0.380305980152
3 -4.02025308637e-09
4 -46.3817306861
5 -50.4689758847
6 0.000107104474599
7 -59.0855170595
8 0.11389455123
9 -74.0569938098
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 227.3148148148148 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
<class 'float'>
current 91.20370370370371 spikes 1
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 398.84259259259255 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
<class 'float'>
current 4.398148148148149 spikes 1
{}
<class '

{}
<class 'sciunit.scores.ZScore'>


{}


...

{}


<class 'float'>
398.84259259259255 vanilla {'injected_square_current': {'duration': '1000.0', 'delay': '100.0', 'amplitude': '227.3148148148148'}} (1, <utilities.VirtualModel object at 0x7f355fe7dac8>, 398.84259259259255)
{}
{}
{}


{}
<class 'sciunit.scores.ZScore'>


{}


{}


In [ ]:
#gap.NGEN
#print(vmpop,pop,invalid_ind,pf)
import pickle
import pandas as pd


try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    '{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False))
    ground_error = list(futures.map(util.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))
ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

#Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = some_tupeles[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])


    

In [ ]:
path

In [ ]:


df_0

In [ ]:
df_1
